In [1]:
from qick import *
import matplotlib.pyplot as plt
import numpy as np

import os
import time
from pynq import Overlay, DefaultIP, PL

In [ ]:
!pip3 install --no-index --no-build-isolation -e ../jupyter_notebooks/qickquack_dir

In [2]:
!pip3 show qickquack

Name: qickquack
Version: 0.0.1
Summary: An extension for qick to use custom DACs
Home-page: https://github.com/hrl-labs-clinic-24-25/qickquack
Author: HMC HRL Clinic Team 24-25
Author-email: UNKNOWN
License: UNKNOWN
Location: /home/xilinx/jupyter_notebooks/qickquack_dir
Requires: qick
Required-by: 


To install qickquack, you MUST run the 'qickquack_installtion' notebook under /jupyter_notebooks/qickquack_dir

In [3]:
import qickquack
help(qickquack)

Help on package qickquack:

NAME
    qickquack

PACKAGE CONTENTS
    AxiPvpGen
    DAC

FILE
    /home/xilinx/jupyter_notebooks/qickquack_dir/qickquack/__init__.py




In [4]:
from qickquack.AxiPvpGen import AxiPvpGen
from qickquack.DAC import DAC, volt2reg

Need to re-define bitfile_path() so that we can use our new bit file

In [5]:
def _new_bitfile_path():
    """Choose the default firmware path for this board.

    Parameters
    ----------

    Returns
    -------
    str
        absolute path to the firmware bitfile distributed with the QICK library
    """
    #board2file =  {'ZCU216' :'qick_216.bit','ZCU111' :'qick_111.bit','RFSoC4x2' :'qick_4x2.bit'}
    #filename = board2file[os.environ['BOARD']]
    filename = "qick_216_integrated_5.bit"
    #src = os.path.join(os.path.dirname(qick.__file__), filename)
    src = os.path.join("/home/xilinx/bit_files/qick_216_integrated", filename)
    
    print("using the products of hmc big")
    return src

In [6]:
qick.bitfile_path = _new_bitfile_path

In [7]:
#PL.reset()

soc = QickSoc()
print(soc)
soccfg = QickConfig(soc.get_cfg())

using the products of hmc big
QICK running on ZCU216, software version 0.2.325

Firmware configuration (built Mon Apr 28 21:27:56 2025):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	16 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.838 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 0 is 0_228, on JHC1
	1:	axis_signal_gen_v6 - envelope memory 16384 samples (1.709 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 1 is 1_228, on JHC2
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (3.419 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 2 is 2_228, on JHC1
	3:	axis_signal_gen_v6 - envelope memory 16384 samples (1.709 us)
		fs=9584.640 MHz, fabric=599.040 MHz, 32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 3 is 3_228, on JHC2
	4:	axis_sg_mixmux8_v1 - envelope memory 0 samples (0.000 us)
		fs=6881

In [ ]:
soc.ip_dict.keys()

In [ ]:
## Setting the initial values as defined in my own test bench (zoe)

dac_call = soc.axi_pvp_gen_v7_0

dac_call.set_start('0', volt2reg(5))
dac_call.set_start('1', volt2reg(4))

dac_call.set_step_size('0', volt2reg(0.3))
dac_call.set_step_size('1', volt2reg(0.2))

dac_call.set_demux('0', 2)
dac_call.set_demux('1', 4)

dac_call.set_group('0', 1)
dac_call.set_group('1', 0)

dac_call.set_dwell_cycles(2500)
dac_call.set_readout_cycles(10)

dac_call.set_num_dims(2) ## this is really important, because otherwise all the logic can and will break
dac_call.set_pvp_width(5)

dac_call.set_trigger_source('user') # this makes it so that we have control. The system will now look to the 
                                  #    TRIGGER_USER_REG instead of the pmod_control
    
bias = DAC(soc)
bias.init_DAC(2)
bias.init_DAC(4)

bias.set_DAC(2, 1)
bias.set_DAC(4, 1)

In [10]:
class Gvg(asm_v2.AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=cfg['nqz'], mixer_freq=0)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])
       
        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch,
                              outsel='product'
                              )
        self.add_pulse(ch=gen_ch, name="sourcedrain", ro_ch=ro_ch,
                       style="const",
                       freq=cfg['freq'],
                       length=cfg['pulse_len'],
                       phase=0,
                       phrst = 1,
                       gain=cfg['gain'],
                      ) #<-pulse length is most of the dwell time
        self.send_readoutconfig(ch=cfg['ro_ch'], name="myro", t=0)
        self.add_loop("myloop", cfg['loops']) ### just for executing the loop 100 times, this should correspond to the number of steps in one row of the GvG
        
        ### setup QuACK dacs here! ###
        bias = DAC(soc)
        daccfg = {
            'startvals': [4.5, 2.25, 3, 0.404],
            'stepsizes': [0.5, 0.5, 0.5, 0.1],
            'demuxvals': [2, 4, 12, 13],
            'groups': [0, 0, 1, 2],
            'mode': 0,
            'width': 4,
            'num_dims': 2 #num_dims is the number of groups, NOT the number of dacs (you can have multiple dacs per group)
            }
        
       # these DAC instructions can't be read unless we are in user mode first
        soc.axi_pvp_gen_v7_0.set_dwell_cycles(3000) #dwell cycles (#cycles, not measured in seconds) should always be less than one period of the qick trigger, so that ldac flips before the next awg and doesn't bottleneck
        soc.axi_pvp_gen_v7_0.set_trigger_source('user')
        #soc.axi_pvp_gen_v7_0.set_user_trigger(0)
        
        for dac in range (len(daccfg['startvals'])):
            bias.init_DAC(daccfg['demuxvals'][dac], debug = 1)
            
            bias.set_DAC(daccfg['demuxvals'][dac], 0, debug = 1) # we don't need to set these because pvp_gen block sets them as its first order of business
            
            
       
        # sets all the registers for this experiment
        soc.axi_pvp_gen_v7_0.setup_pvp(cfg = daccfg)
        
        # send just the first step of pvp manually in order to initialize all DACs to the starting value
        #soc.axi_pvp_gen_v7_0.one_pvp_step()
        
        # go back to waiting for qick to trigger steps
        #soc.axi_pvp_gen_v7_0.set_mode(0)
        soc.axi_pvp_gen_v7_0.set_trigger_source('qick')
        
        self.trigger(ros=[cfg['ro_ch']], pins=[7], t=cfg['trig_time'])#<-setup the trigger for the slow DACs, set to pin0 on PMOD0
        self.delay(t=cfg['meas_delay']) # insert a delay to ensure all the registers are set
       
    def _body(self, cfg):
       
      
        self.delay(t=cfg['meas_delay']) # wait a few us before measuring
        self.pulse(ch=cfg['gen_ch'], name="sourcedrain", t=0) # readout pulse
        self.trigger(ros=[cfg['ro_ch']], t=0, pins=[7]) # trigger ADC for readout, pins=[7] also indicates PMOD0 pin trigger, change the index for other pins
        self.delay_auto(t=cfg['meas_delay']) # this can probably be zero, I add in extra timing slack here
 
 
config = {
          'gen_ch': 14,
          'ro_ch': 0,
          'freq': 1,
          'nqz': 1,
          'trig_time': 0,
          'ro_len': 5,
          'pulse_len': 5,
          'gain': 1,
          'meas_delay': 50,
          'loops': 4 #<-number of steps in a single row, in your case upto 256
          }



# the 7th trigger disappears when it gets connected to the pvp gen block, so add it back in if it's missing
pin_list = soccfg['tprocs'][0]['output_pins'] 
if (len(pin_list) < 8):
    pin_list.append(('trig', 7, 0, 'trigger_pmod'))
                          
gvgprog = Gvg(soccfg, reps=4, final_delay=0, cfg=config) #make plot square by setting all dims to same width



initializing dac 2
setting dac 2 to 0
initializing dac 4
setting dac 4 to 0
initializing dac 12
setting dac 12 to 0
initializing dac 13
setting dac 13 to 0


In [12]:
d = gvgprog.acquire(soc)

  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
soccfg['tprocs'][0]['output_pins'] 

In [ ]:
bias = DAC(soc)
bias.init_DAC(2)
bias.set_DAC(2, 1.5)